In [1]:
import os
from dotenv import load_dotenv
from src.database_manager_factory import DatabaseManagerFactory

In [2]:
load_dotenv()

config = {'db_name': os.getenv("DB_NAME"),
              'db_username': os.getenv("DB_USERNAME"),
              'db_password': os.getenv("DB_PASSWORD"),
              'db_system': os.getenv("DB_SYSTEM"),
              'db_config_file_path': os.getenv("DB_CONFIG_PATH"),
              'db_queries_file_path': os.getenv("DB_QUERIES_PATH"),
              }

db = DatabaseManagerFactory.get_database_manager(username=config['db_username'],
                                                 database_system=config['db_system'],
                                                 database_name=config['db_name'],
                                                 config_file_path=config['db_config_file_path'],
                                                 queries_file_path=config['db_queries_file_path'])

In [3]:
collections = db.get_collections()
collections

[{'id': '3633205',
  'name': 'generalise',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633196',
  'name': 'has_label',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633187',
  'name': 'has_rgb',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633178',
  'name': 'has_generalised',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633169',
  'name': 'has_neuro_column',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633160',
  'name': 'has_non_hebbian_edge_types',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633151',
  'name': 'has_short_term_memory',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633144',
  'name': 'amfabrics_named_query',
  'system': False,
  'type': 'document',
  'status': 'loaded'},
 {'id': '3633135',
  'name': 'has_stats',
  'system': False,
  'type': 'edge',
  'status': 'loaded'},
 {'id': '3633128',
  'nam

In [4]:
for col in collections:
    db.delete_collection(col['name'])

ArangoDBDatabaseManager.delete_create_collection: generalise
ArangoDBDatabaseManager.delete_create_collection: has_label
ArangoDBDatabaseManager.delete_create_collection: has_rgb
ArangoDBDatabaseManager.delete_create_collection: has_generalised
ArangoDBDatabaseManager.delete_create_collection: has_neuro_column
ArangoDBDatabaseManager.delete_create_collection: has_non_hebbian_edge_types
ArangoDBDatabaseManager.delete_create_collection: has_short_term_memory
ArangoDBDatabaseManager.delete_create_collection: amfabrics_named_query
ArangoDBDatabaseManager.delete_create_collection: has_stats
ArangoDBDatabaseManager.delete_create_collection: colour
ArangoDBDatabaseManager.delete_create_collection: rgb
ArangoDBDatabaseManager.delete_create_collection: colour_interest
ArangoDBDatabaseManager.delete_create_collection: NeuroColumn
ArangoDBDatabaseManager.delete_create_collection: AMFabric
